sbpy.data.Ephem Example Notebooks
=================================

[Ephem](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem) provides functionality to query, calculate, manipulate, and store ephemerides and observational information.

Querying Asteroid Ephemerides from JPL Horizons
---------------------------------------

Query ephemerides for three asteroids in a given date range as observed from Maunakea (observatory code 568) using [sbpy.data.Ephem.from_horizons](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_horizons). This function uses [astroquery.jplhorizons](https://astroquery.readthedocs.io/en/latest/jplhorizons/jplhorizons.html) in the background and creates Ephem objects from the query results:

In [1]:
from sbpy.data import Ephem

targets = ['2100', '2018 RC1', 'Ganymed']
eph = Ephem.from_horizons(targets, location=568, 
                          epochs={'start': '2018-01-01', 'stop': '2018-02-01', 'step':'1h'})

eph.table

targetname,datetime_str,datetime_jd,H,G,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,timescale
,,d,mag,,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,,,mag,mag,mag / arcsec2,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str24,str17,float64,float64,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3
2100 Ra-Shalom (1978 RA),2018-Jan-01 00:00,2458119.5,16.05,0.12,*,,298.39175,-15.55282,298.63771,-15.5047,134.878,14.838,190.6996,54.0982,853.05,-156.97,57021.65,27057.68,68.218,20.3416632868,1.233,0.134,18.53,4.31,97.902,3.4e-05,64368.56,*,0.001631,0.0,0.0,0.0,0.0,242.49,0.0,0.0,0.0,313.5825,9.6866,1.052945376618,8.9409616,1.9445240162932,11.5251473,16.172113,24.87625,54.39145,17.8802,/T,16.6621,147.9,98.1,145.4537,62.577,267.646,7.52998,Sgr,69.18394,297.6297498,5.2260156,0.0,0.0,25.501757,-20.596366,13.5798077552,0.000355,0.024,0.009,0.026,0.004,-18.51,0.0,0.026,16.5294,7.5e-06,0.04,0.15,0.00011,145.6446,0.432482309,16.657,305.4411,7.5319,UTC
2100 Ra-Shalom (1978 RA),2018-Jan-01 01:00,2458119.541666667,16.05,0.12,*,,298.43064,-15.54874,298.67659,-15.50057,134.91,14.93554,212.434,48.8381,739.13,-453.94,57001.51,27060.06,68.206,21.3444013055,1.327,0.145,18.53,4.31,97.904,3.4e-05,64350.74,*,0.001631,0.0,0.0,0.0,0.0,242.48,0.0,0.0,0.0,313.6126,9.68,1.0531604447,8.9340224,1.94480261234295,11.6279919,16.17443,24.86934,54.43705,17.8752,/T,16.6541,148.6,98.3,145.4667,62.569,267.641,7.5269,Sgr,69.183941,297.6674694,5.2228933,0.0,0.0,25.521794,-20.629057,14.5795191068,0.000355,0.024,0.009,0.026,0.004,-18.517,0.0,0.026,16.5245,7.5e-06,0.04,0.15,0.00011,145.675,1.432628322,16.6489,305.475,7.5269,UTC
2100 Ra-Shalom (1978 RA),2018-Jan-01 02:00,2458119.583333333,16.05,0.12,*,,298.46956,-15.54464,298.71549,-15.49642,135.0128,15.02554,228.1056,39.6563,596.08,-630.15,56981.11,27062.3,68.195,22.3471393248,1.564,0.17,18.53,4.31,97.906,3.4e-05,64332.6,*,0.00163,0.0,0.0,0.0,0.0,242.47,0.0,0.0,0.0,313.6427,9.6734,1.053375345735,8.9270836,1.94508357085811,11.7203264,16.176767,24.86243,54.50724,17.8702,/T,16.6461,149.4,98.5,145.4798,62.561,267.636,7.52381,Sgr,69.183941,297.7052066,5.2197856,0.0,0.0,25.541865,-20.661751,15.579223717,0.000355,0.024,0.009,0.026,0.004,-18.524,0.0,0.026,16.5197,7.5e-06,0.04,0.15,0.00011,145.7054,2.432773448,16.6409,305.509,7.522,UTC
2100 Ra-Shalom (1978 RA),2018-Jan-01 03:00,2458119.625,16.05,0.12,*,m,298.50851,-15.54051,298.75441,-15.49223,135.1767,15.10336,238.9077,28.2827,476.25,-724.98,56960.4,27064.45,68.183,23.3498773448,2.102,0.229,18.53,4.

The resulting `Ephem` object provides the same functionality as all other `sbpy.data` objects; for instance, you can easily convert values to other units:

In [2]:
import numpy as np
import astropy.units as u

eph['RA'][0].to('arcsec')

<Quantity 1074210.3 arcsec>

Querying Satellite Ephemerides from JPL Horizons (and other ambiguous queries)
------------------------------------------------

Query ephemerides for satellite `'GOES-1'` for a number of epochs as observed from Magdalena Ridge Observatory (observatory code `'H01'`):

In [3]:
eph = Ephem.from_horizons('GOES-1', epochs=[2453452.123245, 2453453.34342, 2453454.342342])

ValueError: Unknown target (GOES-1). Maybe try different id_type?

This function call raises a `ValueError`. The reason for this error is that astroquery.jplhorizons assumes that the provided target string is a small body. However, GOES-1 is an artifical satellite. As suggested in the provided error message, we need to specificy an `id_type` that is different from the default value (`'smallbody'`). Since sbpy.data.Ephem.from_horizons builds upon astroquery.jplhorizons, we can provide optional parameters for the latter directly to the former. According to the [JPL Horizons documentation](https://ssd.jpl.nasa.gov/?horizons_doc#selection), spacecraft are grouped together with major body, so let's try:

In [4]:
eph = Ephem.from_horizons('GOES-1', id_type='majorbody', epochs=[2453452.123245, 2453453.34342, 2453454.342342])

ValueError: Ambiguous target name; provide unique id:
  ID#      Name                               Designation  IAU/aliases/other   
  -------  ---------------------------------- -----------  ------------------- 
  -108366  GOES-1 (spacecraft)                                                  
  -135491  GOES-14 (spacecraft)                                                 
 

Another error. This time, the error refers to the target name being ambiguous and provides a list of objects in the database that match the search string. We pick `GOES-1` and use the unique id provided for this object (note that in this case we have to use `id_type='id'`):

In [5]:
eph = Ephem.from_horizons('-108366', id_type='id', epochs=[2453452.123245, 2453453.34342, 2453454.342342])
eph.table

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat,timescale
,,d,,,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,,,mag,mag,mag / arcsec2,%,arcsec,arcsec,,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,,deg,deg,%,deg,deg,deg,deg,,s,deg,deg,deg,deg,deg,deg,,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,,deg,deg,deg,
str29,str24,float64,str1,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,str3
GOES-1 (spacecraft) (-108366),2005-Mar-22 14:57:28.368,2453452.123245,,,292.53054,-13.56267,292.60033,-13.55515,53839.88,5312.911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,999,0.0,0.0,0.0,33.011,0.0,0.0,--,0.0,0.0,0.0,0.0,0.0,84.09,0.0,0.0,0.0,182.0342,0.0022,0.996462370793,-2.4006489,0.00028198033874,7.54e-05,0.002345,28.94338,3.07344,70.1227,/L,109.8618,145.1,89.7,0.0,264.115,52.735,-8.48354,Sgr,64.185623,292.173953,8.1630192,0.0,0.0,24.955623,-14.610802,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,109.8638,236.6495,7.1011,UTC
GOES-1 (spacecraft) (-108366),2005-Mar-23 20:14:31.488,2453453.34342,,,11.72219,3.09356,11.78241,3.1196,52437.73,13395.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,999,0.0,0.0,0.0,0.608,0.0,0.0,--,0.0,0.0,0.0,0.0,0.0,258.51,0.0,0.0,0.0,183.2266,-0.0005,0.996620933601,0.9627722,0.00028194287078,-0.0005526,0.002345,26.88044,3.07419,8.9376,/T,171.0605,147.7,95.8,0.0,78.509,65.731,1.94249,Psc,64.185629,12.0529652,-1.7868197,0.0,0.0,120.666256,-59.760301,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,171.055,97.7863,-11.5368,UTC
GOES-1 (spacecraft) (-108366),2005-Mar-24 20:12:58.349,2453454.342342,,,12.1502,3.20556,12.21043,3.23158,52443.95,13372.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,999,0.0,0.0,0.0,0.539,0.0,0.0,--,0.0,0.0,0.0,0.0,0.0,259.73,0.0,0.0,0.0,184.2166,-0.0005,0.996902159044,0.9379062,0.00028194166434,-0.0005518,0.002345,26.8658,3.0742,8.4107,/T,171.5875,159.7,98.8,0.0,79.723,65.779,1.99614,Psc,64.185633,12.4904318,-1.8500582,0.0,0.0,121.519585,-59.660567,--,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,--,171.582,98.5263,-12.7094,UTC


Using Ephem objects
-------------------

... please refer to [this notebook](https://github.com/NASA-Planetary-Science/sbpy-tutorial/blob/master/notebooks/data/General_concepts.ipynb) for some examples.

Computing Ephemerides with OpenOrb
-------------------------------

`sbpy.data.Ephem.from_oo` provides a way to compute ephemerides from an [sbpy.data.Orbit](https://github.com/NASA-Planetary-Science/sbpy-tutorial/blob/master/notebooks/data/General_concepts.ipynb) object. This function requires [pyoorb](https://github.com/oorb/oorb/tree/master/python) to be installed.

However, you can use the following code snippet on your computer locally to compute ephemerides for a number of orbits that were obtained from the JPL Horizons system as observed from he Discovery Channel Telescope (observatory code `'G37'`):

In [6]:
from sbpy.data import Ephem, Orbit
from astropy.time import Time
from numpy import arange

orbits = Orbit.from_horizons(['12893', '3552', '2018 RC3'])
epochs = Time.now().jd + arange(0, 3, 1/24)
eph = Ephem.from_oo(orbits, epochs, location='G37')

eph.table

targetname,MJD,RA,DEC,RA_rate,DEC_rate,alpha,elong,r,Delta,V,pa,TopEclLon,TopEclLat,OppTopEclLon,OppTopEclLat,HelEclLon,HelEclLat,OppHelEclLon,OppHelEclLat,EL,ELsun,ELmoon,lunarphase,lunarelong,x,y,z,vx,vy,vz,obsx,obsy,obsz
,d,deg,deg,deg / d,deg / d,deg,deg,AU,AU,mag,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,,deg,AU,AU,AU,AU / d,AU / d,AU / d,AU,AU,AU
str26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
12893 Mommert (1998 QS55),58409.7569293119,133.59813672935624,15.206373546131065,0.25376598339323225,-0.08162646044239083,19.509169529202534,73.31948219302429,2.8574816219418158,2.9793714339129713,19.535114917673837,107.8309255872122,131.75260060683533,-2.139323156263839,106.69248632440747,-2.1394864509018205,112.22931154152235,-2.2305670800893647,87.16919725909449,-2.230730374727346,37.65839591243318,43.35633891066735,-44.17688389669341,0.688092469279113,173.3107203799004,-1.080207336794343,2.643101280809657,-0.11121577519963033,-0.009617242184898966,-0.0031631676198254884,9.057933595648282e-05,0.90241965955896,0.42195845180019387,2.8391928750814976e-06
12893 Mommert (1998 QS55),58409.79859597841,133.60911235142146,15.202969603700183,0.25462048070358617,-0.08174516367097444,19.511845937337913,73.34921396345841,2.8575110488069506,2.978855590608938,19.53484626650216,107.79907604273923,131.7637273293328,-2.1396887148909394,106.66272816648156,-2.14005399833907,112.23775592193653,-2.230468335530591,87.1367567590853,-2.230833618978722,25.308321601526988,45.626159737847175,-32.32089643608873,0.6924279235924838,172.77546859127602,-1.0806080719726556,2.6429694436360465,-0.11121199956616357,-0.009616671143402304,-0.003164564270628255,9.063811304521195e-05,0.9021064124181231,0.4225967060352306,6.351074809646251e-06
12893 Mommert (1998 QS55),58409.84026264539,133.62012716099588,15.199562175447396,0.25557750547130487,-0.0817926358151653,19.51456012412706,73.37890076127499,2.8575404747305746,2.978340470543213,19.53457927303953,107.74626472884779,131.7748915861718,-2.140046554460317,106.63301576939506,-2.140592084137498,112.24620012645086,-2.2303695446098692,87.10432430967413,-2.2309150742870503,12.997203492529263,43.45097210235629,-20.40646538193661,0.6964135400080278,172.27214007885902,-1.0810087833161435,2.642837548278588,-0.11120822148389209,-0.009616099907814299,-0.0031659608085547646,9.069688631743625e-05,0.9017949104063602,0.42323560095680585,9.484859608027696e-06
12893 Mommert (1998 QS55),58409.8819293119,133.63118400197442,15.196154338009999,0.25656301883302013,-0.0817645042699087,19.517313924143057,73.40857463418811,2.857569899713812,2.9778254834541307,19.534313565228018,107.67664859203992,131.7860951729564,-2.1403929559961057,106.60331750240032,-2.1410846228566727,112.25464415557718,-2.2302707073263637,87.07186648502109,-2.2309623741869307,0.9771591469885361,37.38718629134101,-8.652440041359437,0.700027514299517,171.8084334733252,-1.0814094708405095,2.6427055947341622,-0.11120444095275106,-0.009615528478125352,-0.0031673572336646427,9.075565577560296e-05,0.901484833636355,0.42387563354047214,1.202557299476159e-05
12893 Mommert (1998 QS55),58409.92359597841,133.64228246056027,15.192749242818781,0.25750100403868087,-0.08166156633552428,19.520106118634338,73.43826893924467,2.857599323758837,2.9773100237019303,19.534048662546255,107.5954383221994,131.79733683904885,-2.1407249917913935,106.57360034235762,-2.1415186575982443,112.26308801012956,-2.230171823675676,87.03935151343832,-2.230965489482527,-10.477011230300107,28.598648024220687,2.72482736988716,0.7032705701601674,171.3883643352497,-1.0818101345757969,2.642573582994906,-0.11120065797253886,-0.009614956854305369,-0.0031687535460674767,9.081442142426185e-05,0.9011757325750132,0.4245172076401475,1.3798874306109044e-05
12893 Mommert (1998 QS55),58409.96526264539,

We can compare the calculated ephemerides to ephemerides provided by JPL Horizons to check their accuracy by calculating the maximum absolute residuals in RA, Dec, and heliocentric distance (r):

In [7]:
import numpy as np
eph_horizons = Ephem.from_horizons(['12893', '3552', '2018 RC3'], epochs=epochs, location='G37')

print('RA max residual:', np.max(np.fabs(eph['RA']-eph_horizons['RA']).to('arcsec')))
print('DEC max residual', np.max(np.fabs(eph['DEC']-eph_horizons['DEC']).to('arcsec')))
print('r max residual:', np.max(np.fabs(eph['r']-eph_horizons['r']).to('km')))

RA: 0.018232642582916014 arcsec
DEC 0.01812717567730715 arcsec
r: 13.00534292085884 km


Querying Ephemerides from the Minor Planet Center
======================

Ephemerides from the [Minor Planet Center](http://minorplanetcenter.net) can be queried using the function [sbpy.data.Ephem.from_mpc()](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_mpc), which uses [astroquery.mpc](https://astroquery.readthedocs.io/en/latest/mpc/mpc.html) under the hood and uses the same syntax as [sbpy.data.Ephem.from_horizons](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.Ephem.html#sbpy.data.Ephem.from_horizons). 

The following example compares ephemerides for asteroid Ceres for a given date and observer location from JPL Horizons and the Minor Planet Center:

In [3]:
from sbpy.data import Ephem
import numpy as np

ceres_horizons = Ephem.from_horizons('Ceres', epochs=2451200.5, location=500)
ceres_mpc = Ephem.from_mpc('Ceres', epochs=2451200.5, location=500)

print('RA residual:', np.max(np.fabs(ceres_mpc['RA']-ceres_horizons['RA']).to('arcsec')))
print('DEC residual', np.max(np.fabs(ceres_mpc['DEC']-ceres_horizons['DEC']).to('arcsec')))
print('r residual:', np.max(np.fabs(ceres_mpc['r']-ceres_horizons['r']).to('km')))

RA residual: 0.22799999996152565 arcsec
DEC residual 0.03200000000020964 arcsec
r residual: 49377.70036691681 km
